### Regresión logística para clasificación binaria

- La regresión logística se utiliza para problemas de clasificación.
- La regresión logística genera probabilidades.

#### Reglas probabilísticas:

- Si la probabilidad p > 0,5, entonces los datos se etiquetan como 1.
- Si la probabilidad p < 0,5, entonces los datos se etiquetan como 0.

#### Regresión logística en Scikit-learn

```python
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

logreg = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
```

#### Predecir probabilidades

```python
y_pred_probs = logreg.predict_proba(X_test)[:,1]
print(y_pred_probs[0])
```

### Umbrales de probabilidad

- Por defecto, el umbral de regresión logística es 0,5.
- No es específico de la regresión logística.
- Los clasificadores KNN también tienen umbrales.

¿Qué ocurre si variamos el umbral?

### Curva ROC

**ROC (Receiver Operating Characteristic)** es una curva que muestra el rendimiento de un modelo de clasificación binaria al variar el umbral de decisión.

- **Eje X:** Tasa de Falsos Positivos (FPR)
- **Eje Y:** Tasa de Verdaderos Positivos (TPR o Recall)

**¿Para qué sirve?**

- Permite visualizar la capacidad del modelo para distinguir entre clases.
- Ayuda a comparar distintos modelos de clasificación.

```python
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()
```

### ROC AUC

**AUC (Area Under the Curve)** es una métrica que resume el rendimiento del modelo en una sola cifra entre `0` y `1`.

- **AUC = 1.0:** El modelo clasifica perfectamente.
- **AUC = 0.5:** El modelo no tiene capacidad predictiva (equivale a lanzar una moneda).

**¿Para qué sirve?**

- Mide la habilidad del modelo para diferenciar entre clases positivas y negativas.
- Es especialmente útil en casos de clases desbalanceadas.

```python
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred_probs))
```

### Ejemplo

En la actualidad, la diabetes representa uno de los principales problemas de salud pública a nivel mundial, afectando a millones de personas y generando costos significativos para los sistemas de salud. Una detección temprana y precisa puede marcar la diferencia en el tratamiento y la calidad de vida de los pacientes. Sin embargo, los métodos tradicionales de diagnóstico suelen ser tardíos o requerir procedimientos invasivos y costosos.

Frente a este contexto, surge la necesidad de explorar herramientas analíticas que permitan predecir con antelación la presencia de diabetes a partir de variables clínicas y demográficas fácilmente obtenibles. Esto plantea el siguiente interrogante:

¿Es posible construir un modelo predictivo, basado en regresión logística, que utilice variables clínicas para estimar con precisión la probabilidad de que un paciente tenga diabetes?

Este trabajo utiliza un conjunto de datos clínicos preprocesados y aplica técnicas de escalado y partición de datos, junto con un modelo de regresión logística, para entrenar y evaluar un clasificador. A través de métricas como la probabilidad estimada, la matriz de confusión y el valor del ROC AUC, se busca determinar la capacidad del modelo para asistir en decisiones médicas tempranas con base en la predicción de la condición diabética de los pacientes.

In [24]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

diabetes_df = pd.read_csv("../data/diabetes_clean.csv")
X = diabetes_df.drop("diabetes", axis=1).values
y = diabetes_df["diabetes"].values

"""
Ese mensaje es un ConvergenceWarning, lo lanza scikit-learn cuando el algoritmo de regresión logística no logra converger
en el número máximo de iteraciones (max_iter=100 por defecto). Esto significa que el modelo no ha encontrado una solución
óptima y probablemente tus resultados no sean confiables aún.
"""

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=21)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred_probs = logreg.predict_proba(X_test)[:, 1]

print(y_pred_probs[:10])
print(diabetes_df.head(5))

[0.27060208 0.08157744 0.07884016 0.59335757 0.52139707 0.1228702
 0.06507872 0.05328381 0.03461993 0.13238987]
   pregnancies  glucose  diastolic  triceps  insulin   bmi    dpf  age  \
0            6      148         72       35        0  33.6  0.627   50   
1            1       85         66       29        0  26.6  0.351   31   
2            8      183         64        0        0  23.3  0.672   32   
3            1       89         66       23       94  28.1  0.167   21   
4            0      137         40       35      168  43.1  2.288   33   

   diabetes  
0         1  
1         0  
2         1  
3         0  
4         1  


Estas son las probabilidades predichas de que los primeros 10 pacientes del conjunto de prueba tengan diabetes. Valores cercanos a 0 indican baja probabilidad, y valores cercanos a 1 indican alta probabilidad.

In [26]:
print(roc_auc_score(y_test, y_pred_probs))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8123403575989783
[[131  13]
 [ 48  39]]
              precision    recall  f1-score   support

           0       0.73      0.91      0.81       144
           1       0.75      0.45      0.56        87

    accuracy                           0.74       231
   macro avg       0.74      0.68      0.69       231
weighted avg       0.74      0.74      0.72       231



### Interpretación de Metricas
#### ROC AUC

```
ROC AUC = 0.812
```

**Qué indica:** El modelo tiene buena capacidad discriminativa.

**Interpretación:** Hay un 81.2% de probabilidad de que el modelo clasifique correctamente una persona con diabetes frente a una sin diabetes.

**Valores aceptables:**
- 0.80: Bueno
- 0.90: Excelente

#### Matriz de Confusión
```
[[131  13]
 [ 48  39]]
```

- Verdaderos negativos (VN): 131
- Falsos positivos (FP): 13
- Falsos negativos (FN): 48
- Verdaderos positivos (VP): 39

**Conclusión:** El modelo comete más errores al no detectar personas con diabetes (FN = 48) que al clasificarlas como diabéticas cuando no lo son (`FP = 13`).

### Reporte de Clasificación

#### Conclusiones:

- El modelo detecta bien a los no diabéticos (91% de recall), pero tiene dificultades con los diabéticos (solo 45% de recall).
- Esto sugiere que el modelo es conservador: prefiere evitar falsos positivos, a costa de cometer más falsos negativos.